In [2]:
# This notebook will be the test bed for data read functions to ingest data from 
# a data folder on the local machine

In [5]:
# imports

import pandas as pd
import numpy as np
import os

In [6]:
# set path to WAPO file and txt files

wapo = "/Users/andrewkroening/Desktop/720_Data/arcos_all_washpost.tsv.gz"
nick_path = "/Users/andrewkroening/Desktop/720_Data/US_VitalStatistics"

In [4]:
# Ingest the WAPO file as chunks

chunks = 500000

states = ["FL", "TX", "WA", "GA", "OK", "OR"]

keep_df = pd.DataFrame()

for chunk in pd.read_csv(wapo, sep='\t', compression='gzip', chunksize = chunks, usecols=["BUYER_COUNTY", "BUYER_STATE", "DRUG_NAME", "TRANSACTION_DATE", "QUANTITY", "UNIT"]):
    if chunk["BUYER_STATE"].isin(states).any():
        chunk["COUNTY_NAME"] = chunk["BUYER_COUNTY"] + " COUNTY, " + chunk["BUYER_STATE"]
        chunk["TRANSACTION_DATE"] = pd.to_datetime(chunk["TRANSACTION_DATE"], format="%m%d%Y")
        chunk["YEAR"] = chunk["TRANSACTION_DATE"].dt.year
        chunk["MONTH"] = chunk["TRANSACTION_DATE"].dt.month
        chunk = chunk.groupby(["COUNTY_NAME", "YEAR", "MONTH"]).agg({"QUANTITY": "sum"}).reset_index()

        keep_df = pd.concat([keep_df, chunk])


/var/folders/79/hh7cth4s3p3dbtdpc8j1m9bm0000gn/T/ipykernel_18832/2821673315.py:7: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(wapo, sep='\t', compression='gzip', chunksize = chunks, usecols=["BUYER_COUNTY", "BUYER_STATE", "DRUG_NAME", "TRANSACTION_DATE", "QUANTITY", "UNIT"]):
/var/folders/79/hh7cth4s3p3dbtdpc8j1m9bm0000gn/T/ipykernel_18832/2821673315.py:7: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(wapo, sep='\t', compression='gzip', chunksize = chunks, usecols=["BUYER_COUNTY", "BUYER_STATE", "DRUG_NAME", "TRANSACTION_DATE", "QUANTITY", "UNIT"]):
/var/folders/79/hh7cth4s3p3dbtdpc8j1m9bm0000gn/T/ipykernel_18832/2821673315.py:7: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(wapo, sep='\t', compression='gzip', chunksize = chunks, usecols=["BU

In [ ]:
keep_df.sample(10)

In [52]:
# subset the df to the columns we are most interested in
# df2 = df[["BUYER_COUNTY", "BUYER_STATE", "DRUG_NAME", "TRANSACTION_DATE", "QUANTITY", "UNIT"]].copy()
df2 = df.copy()
df2["COUNTY_NAME"] = df2["BUYER_COUNTY"] + " COUNTY, " + df2["BUYER_STATE"]
df2["TRANSACTION_DATE"] = pd.to_datetime(df2["TRANSACTION_DATE"], format="%m%d%Y")
df2["YEAR"] = df2["TRANSACTION_DATE"].dt.year
df2["MONTH"] = df2["TRANSACTION_DATE"].dt.month
df2 = df2.groupby(["COUNTY_NAME", "YEAR", "MONTH"]).agg({"QUANTITY": "sum"}).reset_index()
df2.sample(10)

,COUNTY_NAME,YEAR,MONTH,QUANTITY
784,"CHENANGO COUNTY, NY",2008,5,2.0
4150,"SUFFOLK COUNTY, NY",2012,2,1.0
2907,"MORRIS COUNTY, NJ",2007,11,1.0
2803,"MITCHELL COUNTY, GA",2011,12,1.0
4927,"YATES COUNTY, NY",2012,5,1.0
1457,"ESSEX COUNTY, NJ",2006,2,5.0
3957,"SCHOHARIE COUNTY, NY",2009,7,2.0
608,"CALHOUN COUNTY, AL",2011,11,1.0
562,"CAGUAS COUNTY, PR",2006,9,1.0
1846,"GRADY COUNTY, GA",2007,8,1.0


In [42]:
# count rows in df2 where UNIT is not NaN
df2["UNIT"].count()

0

In [7]:
# generate a df from the txt files in a folder path

def reader(path):
    '''Function to read a series of txt files in a directory and return a single df
    
    Args:
        path: path to the directory containing the txt files
        
    Returns:
        nick_df: a single df containing all the data from the txt files'''
    
    # initialize the empty df
    nick_df = pd.DataFrame()

    # set a loop to iterate through the files in the folder
    for file in os.listdir(path):
        if file.endswith(".txt"):
            txt_table = pd.read_table(os.path.join(path, file), sep='\t')
            nick_df = pd.concat([nick_df, txt_table], axis=0)
    return nick_df

this_df = reader(nick_path)

In [11]:
this_df.sample(10)

,Notes,County,County Code,Year,Year Code,Drug/Alcohol Induced Cause,Drug/Alcohol Induced Cause Code,Deaths
4292,NaN,"Milwaukee County, WI",55079.0,2009.0,2009.0,Drug poisonings (overdose) Unintentional (X40-...,D1,151.0
62,NaN,"Mobile County, AL",1097.0,2010.0,2010.0,All other non-drug and non-alcohol causes,O9,3967.0
422,NaN,"Alamosa County, CO",8003.0,2010.0,2010.0,All other non-drug and non-alcohol causes,O9,127.0
3066,NaN,"Muskogee County, OK",40101.0,2009.0,2009.0,All other non-drug and non-alcohol causes,O9,807.0
3520,NaN,"Crockett County, TN",47033.0,2010.0,2010.0,All other non-drug and non-alcohol causes,O9,170.0
2125,NaN,"Smith County, MS",28129.0,2011.0,2011.0,All other non-drug and non-alcohol causes,O9,156.0
4318,NaN,"Columbia County, WA",53013.0,2013.0,2013.0,All other non-drug and non-alcohol causes,O9,50.0
1176,NaN,"Orange County, IN",18117.0,2009.0,2009.0,All other non-drug and non-alcohol causes,O9,209.0
2119,NaN,"Missoula County, MT",30063.0,2003.0,2003.0,All other non-drug and non-alcohol causes,O9,678.0
241,NaN,"Little River County, AR",5081.0,2015.0,2015.0,All other non-drug and non-alcohol causes,O9,170
